In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile

zip_path = '/content/drive/MyDrive/cat_dog_data.zip'  # change if inside a folder
extract_to = '/content/cat_dog_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [4]:
!ls /content/cat_dog_dataset/PetImages

Cat  Dog


In [5]:
import os, shutil, random

base_dir = '/content/cat_dog_dataset'
src_dir = os.path.join(base_dir, 'PetImages')

# Create target directories
sets = ['train', 'validation', 'test']
categories = ['cats', 'dogs']

for s in sets:
    for cat in categories:
        os.makedirs(os.path.join(base_dir, s, cat), exist_ok=True)

def split_data(category, label):
    src_folder = os.path.join(src_dir, category)
    images = [img for img in os.listdir(src_folder) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    n = len(images)
    train_split = int(0.7 * n)
    val_split = int(0.15 * n)

    train_imgs = images[:train_split]
    val_imgs = images[train_split:train_split + val_split]
    test_imgs = images[train_split + val_split:]

    for img in train_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'train', label, img))
        except:
            pass

    for img in val_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'validation', label, img))
        except:
            pass

    for img in test_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'test', label, img))
        except:
            pass

# Run for both categories
split_data('Cat', 'cats')
split_data('Dog', 'dogs')

In [6]:
train_dir = '/content/cat_dog_dataset/train'
val_dir = '/content/cat_dog_dataset/validation'
test_dir = '/content/cat_dog_dataset/test'

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen= ImageDataGenerator(rescale=1./255)

In [9]:
train_gen= train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    batch_size= 32,
    class_mode= "binary"
)

valid_gen= val_test_datagen.flow_from_directory(
    val_dir,
    target_size= (150,150),
    batch_size= 32,
    class_mode= "binary"
)

test_gen= val_test_datagen.flow_from_directory(
    test_dir,
    target_size= (150,150),
    batch_size= 32,
    class_mode= "binary",
    shuffle= False
)

Found 17498 images belonging to 2 classes.
Found 3748 images belonging to 2 classes.
Found 3752 images belonging to 2 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),  # Extract basic features (edges, corners)
    MaxPooling2D(2, 2),  # Downsample to reduce spatial size and computation

    Conv2D(64, (3,3), activation='relu'),  # Extract deeper/more complex features
    MaxPooling2D(2, 2),  # Further downsampling

    Conv2D(128, (3,3), activation='relu'), # Learn even higher-level features
    MaxPooling2D(2, 2),  # Reduce size while keeping important features

    Flatten(),  # Convert 3D feature maps to 1D vector for dense layers

    Dense(512, activation='relu'),  # Fully connected layer to learn non-linear combinations
    Dropout(0.5),  # Prevent overfitting by randomly deactivating 50% neurons

    Dense(1, activation='sigmoid')  # Output layer for binary classification (cat or dog)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [12]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=valid_gen
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 140s 244ms/step - accuracy: 0.5165 - loss: 0.7588 - val_accuracy: 0.5576 - val_loss: 0.6818
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 130s 237ms/step - accuracy: 0.5726 - loss: 0.6807 - val_accuracy: 0.6321 - val_loss: 0.6495
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 127s 233ms/step - accuracy: 0.6281 - loss: 0.6464 - val_accuracy: 0.6710 - val_loss: 0.5976
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 128s 234ms/step - accuracy: 0.6694 - loss: 0.6127 - val_accuracy: 0.7463 - val_loss: 0.5276
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 127s 232ms/step - accuracy: 0.6996 - loss: 0.5810 - val_accuracy: 0.7385 - val_loss: 0.5181
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 131s 240ms/step - accuracy: 0.7189 - loss: 0.5486 - val_accuracy: 0.7796 - val_loss: 0.4574
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 126s 231ms/step - accuracy: 0.7562 - loss: 0.5108 - val_accuracy: 0.8004 - val_loss: 0.4485
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 128s 234ms/step - accuracy: 0.7651 -

In [13]:
model.save('/content/drive/MyDrive/cat_dog_model.h5')

In [14]:
history.history['accuracy']

[0.5354897975921631,
 0.5877243280410767,
 0.641444742679596,
 0.6811635494232178,
 0.7056235074996948,
 0.7273974418640137,
 0.7582009434700012,
 0.767859160900116,
 0.7831752300262451,
 0.789747416973114]

In [15]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}")

114/118 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8530 - loss: 0.3517

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


118/118 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - accuracy: 0.8520 - loss: 0.3525
Test Accuracy: 0.8300


In [16]:
print(test_gen.class_indices)

{'cats': 0, 'dogs': 1}


In [16]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions
test_probs = model.predict(test_gen)
test_preds = (test_probs > 0.5).astype(int).reshape(-1)
true_labels = test_gen.classes
class_names = list(test_gen.class_indices.keys())

# Metrics
print(f"✅ Test Accuracy: {test_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, test_preds))
print("\nClassification Report:")
print(classification_report(true_labels, test_preds, target_names=class_names))

118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step
✅ Test Accuracy: 0.8300

Confusion Matrix:
[[1624  252]
 [ 386 1490]]

Classification Report:
              precision    recall  f1-score   support

        cats       0.81      0.87      0.84      1876
        dogs       0.86      0.79      0.82      1876

    accuracy                           0.83      3752
   macro avg       0.83      0.83      0.83      3752
weighted avg       0.83      0.83      0.83      3752

